In [ ]:
from transformers import pipeline
from PIL import Image
import requests
import torch
import numpy as np
from IPython.display import Markdown, display

In [ ]:
# from huggingface_hub import login
# login(token="<>hugging_face_token</>")  # Uncomment this line if you need to log in using Hugging Face CLI

# LOGGED IN USING HUNNGINGFACE CLI

In [ ]:
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
)

In [ ]:
medical_transcript = """
Physician: Good morning, Ms. Johnson. What brings you in today?

Patient: Morning, doctor. I’ve been having this persistent cough for the last two weeks. It started out dry, but now there’s some phlegm.

Physician: I see. Any fever or chills?

Patient: Yes, I had a mild fever three days ago, around 100.5°F, but it went away on its own.

Physician: Any shortness of breath or chest pain?

Patient: Not really chest pain, but my chest feels a bit tight, especially when I’m coughing a lot.

Physician: How about your energy levels or appetite?

Patient: I’ve been more tired than usual, and I haven’t had much of an appetite.

Physician: Any recent travel or contact with anyone who’s been sick?

Patient: No travel, but my son had a cold last week.

Physician: Okay. Let’s take a listen to your lungs.

[Stethoscope exam sounds]

Physician: I’m hearing some mild crackles in your lower lungs. No wheezing, which is good. I’ll check your oxygen level—98%, that’s normal.

Physician: I’d like to order a chest X-ray to rule out pneumonia, and we’ll also do a CBC and test for respiratory viruses. In the meantime, I’ll prescribe an inhaler to help with the chest tightness, and a cough suppressant so you can rest.

Patient: Sounds good. Should I stay home from work?

Physician: Yes, for at least a couple of days until the results come in and you’re feeling better. Stay hydrated, and rest as much as you can.

Patient: Will do. Thanks, doctor.

Physician: You’re welcome. We’ll follow up as soon as we have the test results.
"""

In [ ]:
system_prompt = """
You are a clinical documentation assistant. Your task is to read medical transcripts 
(dialogues between clinicians and patients) and convert them into structured clinical notes using the SOAP format.

Follow these rules:

*S – Subjective*:
Include all information reported by the patient: symptoms, duration, history, complaints, and any relevant lifestyle or 
exposure context. Use the patient’s own words when possible (paraphrased for clarity).

*O – Objective*:
Include observable findings such as vital signs, physical exam results, lab tests, imaging results, and clinician 
observations during the encounter.

*A – Assessment*:
Provide a brief summary of the clinician’s diagnostic impression. Include possible or confirmed diagnoses.

*P – Plan*:
Outline the next steps recommended by the clinician. This can include prescriptions, tests to be ordered, 
referrals, follow-up instructions, and lifestyle recommendations.

Keep the format clear and professional. Do not include any parts of the transcript that are irrelevant or 
non-clinical. Do not invent information not found in the transcript. 
Allways use a bullet point format for each section of the SOAP note.

A comprehensive SOAP note has to take into account all subjective and objective information, 
and accurately assess it to create the patient-specific assessment and plan.

Here is the transcript from a medical record file from which you will be asked to extract relevant SOAP information:

"""

In [ ]:
dummy_image = Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))


In [ ]:
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": system_prompt}]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": medical_transcript},
             {"type": "image", "image": dummy_image}
            
        ]
    }
]

In [ ]:
output = pipe(text=messages, max_new_tokens=512)


In [ ]:
display(Markdown(output[0]["generated_text"][-1]["content"]))


In [ ]:
image_path = "medical_transcript.png"
image = Image.open(image_path)

In [ ]:
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": system_prompt}]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": ""},
            {"type": "image", "image": image},
        ]
    }
]

In [ ]:
output = pipe(text=messages, max_new_tokens=512)


In [ ]:
display(Markdown(output[0]["generated_text"][-1]["content"]))
